# Проект: Защита персональных данных клиентов

**Описание исседования** <br>

Нужно защитить данные клиентов страховой компании «Хоть потоп» и разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. При этом качество моделей машинного обучения не должно ухудшиться. 

**Содержание**

 [Шаг 1. Загрузка и подготовка данныx](#step1)
 
 [Шаг 2. Изменится ли качество линейной регрессии, если признаки умножить на обратимую матрицу](#step2)
 
 [Шаг 3. Алгоритм преобразования данных](#step3)
 
 [Шаг 4. Проверка алгоритма](#step4)
 
 [Общий вывод](#step5)

<a id = 'step1'></a>
# Шаг 1. Загрузка и подготовка данных

In [1]:
#импортируем библиотеки
import pandas as pd
import numpy as np
import os

from sklearn.linear_model import LinearRegression

In [2]:
#считываем файл
if os.path.exists('insurance.csv'):
    data = pd.read_csv('insurance.csv') #локальный путь
else:
    data = pd.read_csv('/datasets/insurance.csv') #путь, указанный в проекте       

In [3]:
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [5]:
#фичи и таргет
X = data.drop('Страховые выплаты', axis=1)
y = data['Страховые выплаты']

<a id = 'step2'></a>
# Шаг 2. Изменится ли качество линейной регрессии, если признаки умножить на обратимую матрицу?

In [6]:
#класс с двумя методами:
#    1.линейная регрессия с обычными признакми
#    2.линейная регрессия с признаками, умноженными на обратимую матрицу

class LinearRegressionTest:
    
    def fit_predict(self, X, y):
            
        model = LinearRegression()
        model.fit(X, y)
        r2 = model.score(X,y)
        return r2
       
    def fit_predict_inv(self, X, y):
        
        matrix = np.random.rand(X.shape[1], X.shape[1])
        
        try:
            matrix_inv = np.linalg.inv(matrix)
            X = X @ matrix_inv
            model = LinearRegression()
            model.fit(X, y)
            r2 = model.score(X,y)
            return r2
            
        except np.linalg.LinAlgError:
            print('You are lucky: singular matrix!','Try again', sep='\n')
            return   

In [7]:
model = LinearRegressionTest()
for i in range(5):
    print('Iter', i)
    print('R2_score:', model.fit_predict(X,y))
    print('R2_score with matrix inverse multiplication:', model.fit_predict_inv(X,y))
    print()

Iter 0
R2_score: 0.42494550286668
R2_score with matrix inverse multiplication: 0.4249455028666823

Iter 1
R2_score: 0.42494550286668
R2_score with matrix inverse multiplication: 0.4249455028666864

Iter 2
R2_score: 0.42494550286668
R2_score with matrix inverse multiplication: 0.4249455028666912

Iter 3
R2_score: 0.42494550286668
R2_score with matrix inverse multiplication: 0.4249455028666874

Iter 4
R2_score: 0.42494550286668
R2_score with matrix inverse multiplication: 0.424945502866644



<div class="alert alert-block alert-info">
    
**Комментарий студента** 
    
Умножая матрицу признаков размера NxM  на любую обратимую матрицу размера MхМ, мы снова получаем матрицу размера NxM, но с другими значениями признаков. Это значит, что изменится и искомый вектор весов, но суть метода от этого не меняется. В этом случаче также будут найдены оптимальные веса, минимизирующие функцию потери, а значит, метрика качества модели не должна пострадать. Что мы и видим на примере класса LinearRegressionTest.
    
Забегая вперед, именно поэтому с матрицей признаков можно делать любые элементарные преобразования.
В моем предложенном алгоритме :
   * исходная матрица умнажается на 10000
   * вычитается 1000
   * умножим результат на 3
   * умножим на случайную обратимую матрицу

<a id = 'step3'></a>
# Шаг 3. Алгоритм преобразования данных

In [8]:
#преобразование
def transformation(X):
    matrix = np.random.rand(X.shape[1], X.shape[1])
    try:
        matrix_inv = np.linalg.inv(matrix)
        return (X * 10000 - 1000) * 3 @ matrix_inv
        
    except np.linalg.LinAlgError:
            print('You are lucky: singular matrix!','Try again', sep='\n')
            return 

In [9]:
transformation(X).head()

,0,1,2,3
0,1.480730e+09,-6.790386e+08,1.307190e+09,-4.737716e+08
1,1.133879e+09,-5.200128e+08,1.002354e+09,-3.629200e+08
2,6.265154e+08,-2.873038e+08,5.541475e+08,-2.005875e+08
3,1.245297e+09,-5.711642e+08,1.098224e+09,-3.982644e+08
4,7.789813e+08,-3.571827e+08,6.882155e+08,-2.493273e+08


<a id = 'step4'></a>
# Шаг 4. Проверка алгоритма

In [10]:
X_new = transformation(X)
m = LinearRegressionTest()

print('R2_score after transformation:', m.fit_predict(X_new,y))

R2_score after transformation: 0.4249455028666611


<a id = 'step5'></a>

<div class="alert alert-block alert-info">
    
<h2> Общий вывод <a class="tocSkip"></h2>
    
В данной работе мы познакомились с данными клиентов страховой компании «Хоть потоп» и на их примере выяснили, что умножение исходной матрицы призаков на случайную обратимую матрицу не влияет на качество линейной регрессии. Более того, на ее качество не влияет любое элементарное преобразование над матрицей с признаками.
    
Исходя из этого предположения, мы разработали алгоритм шифрования данных, основанный на элементарных преобразованиях над матрицей с признаками, который должен помочь скрыть персональную информацию о клиентах компании.
</div>   